## [Introduction](#toc)<a id="intro"></a>

Drying is the thermal removal of water from the ammonium diuranate.
Calcination is the chemical conversion from the ammoinium diuranate to the U3O8 product. The chemical reaction involved with this process is____. 

SOURCES
hearth dryers: https://patentimages.storage.googleapis.com/94/7d/7c/3c2de05ed45669/US5957064.pdf

### [Problem Statement](#toc)<a id="prob"></a>

In [1]:
from cortix import ReactionMechanism

In [2]:
rxn_mech = ReactionMechanism(file_name='evap-triuranium-octoxide-production-mechanism.txt', order_species=True)
rxn_mech.md_print()

5 **Species:** 
 ${\mathrm{H}_{2}\mathrm{O}}_\mathrm{(a)} \quad  {\mathrm{N}_{2}}_{\mathrm{(v)}} \quad  {\mathrm{N}\mathrm{H}_{3}}_{\mathrm{(a)}} \quad  {\mathrm{U}_{3}\mathrm{O}_{8}}_{\mathrm{(s)}} \quad  {[\mathrm{N}\mathrm{H}_{4}]_{2}\mathrm{U}_{2}\mathrm{O}_{7}}_{\mathrm{(a)}}$

1 **Reactions:** 
 \begin{align*} 
9.0\,{[\mathrm{N}\mathrm{H}_{4}]_{2}\mathrm{U}_{2}\mathrm{O}_{7}}_{\mathrm{(a)}}\ &\longrightarrow \ 15.0\,{\mathrm{H}_{2}\mathrm{O}}_\mathrm{(a)}\ + \ 2.0\,{\mathrm{N}_{2}}_{\mathrm{(v)}}\ + \ 14.0\,{\mathrm{N}\mathrm{H}_{3}}_{\mathrm{(a)}}\ + \ 6.0\,{\mathrm{U}_{3}\mathrm{O}_{8}}_{\mathrm{(s)}}\\ 
\end{align*} 


In [4]:
'''Data Check'''
print('Is mass conserved?', rxn_mech.is_mass_conserved())
#rxn_mech.rank_analysis(verbose=True, tol=1e-8)
#print('S=\n', rxn_mech.stoic_mtrx)

Is mass conserved? True


## [Objectives](#toc)<a id="obj"></a>

- Python code will be used to develop a dynamic comprehensive system that accurately represents the evaporation and calcination parts of the milling process as well as their role in simulating a steady state reactor. This involves both mathematical models of the system’s material streams and visual representations of the data on a time dependent basis. 
- Modules to model evaporation and calcination steps will be developed.
- Deliverable content will be prepared using Jupyter notebooks to seamlessly integrate working code with descriptive markdown text.  
- A flowsheet based on White Mesa will be developed as the basis for our modules
- Pick up where the previous group left off.


## [Tools](#toc)<a id="tools"></a>

## [Approach and Dynamics Equations](#toc)<a id="app"></a>

-  First, a series of mass balances and component balances are made based on the desired input of 1 ton of raw material.
- These balances then take into account the chemical kinetics and related rates required in the operation of White Mesa's chemical reactions and fluid flow operations
- Once baseline models of each individual process exist, they are then programmed into 5 separate but connected modules within Cortix
- Each process will be capable of interacting with the modules connected to it on either end, allowing them to share data such as the incoming and outgoing streams from one process to the next

## [Challenges & Course Correctons](#toc)<a id="correct"></a>

## [Initialization of Process Modules](#toc)<a id="IPM"></a>

### [Evaporation/Calcining](#toc)<a id="evap"></a>

```python
                                              |
                                              |  Ammonium Diuranate Feed
                                              v
                                       ________________       
                                      |                |
                                      |                |
                                      |                |<------ Steam Sparging
                                      |     Drying     |
                                      |                |
                   Off-Gas <----------|                |<------ Resistance Heating (Internal)
                                      |                |
                                      |----------------|
                      Off-Gas <-------|                |<------ Sweeping Gas
                                      |                |
                                      |   Calcination  |
                 Sweeping Gas <-------|                |
                   (Disposal)         |                |
                                      |________________|<------ Resistance Heating (Internal)
                                               |
                                               |
                                               |
                                               v
                                        Product (U3O8)
```

__Uranium Drying Overview__

The incoming slurry produced by precipitaiton now has to undergo a drying process at extremly high temperatures. While the centrifuge in the precipitation process drove out most of the water, it needs to be completely evaporated to obtain the final product. The product being fed into the drying section of the milling process mainly contains ammonium diuranate, which is dried to produce the coveted triuranium octoxide. Two stages make up the drying process, drying, which is the thermal removal of water fro the ammonium diurante, and calcination, which is the chemical conversion of ammonium diuranate to U3O8. 

There are two reactions that will happen based off each of the stages. In the drying stage, heat will be applied to the ammonium diurante slurry, resulting in the slurry and water vapor separated. In the calcination statge, the dried ammonium diuranate slurry will be hit with even mroe heat, chemically changing it to the U3O8 product, with some side products as well.

The reaction that is happening in the drying process is typically summerized as one whole reaction since it only accounts for the addition of heat throughout the process. This heat will cause the ammonia compound to seperate from the ammonium diuranate, along with other light impurites and yield the desired product of triuranium octoxide. The reaction of this heating process is as follows:

As a whole, this reaction can be summarized as the following:

*Production of Triuranium Octoxide*

</br>
<center>9(NH<sub>4</sub>)<sub>2</sub>U<sub>2</sub>0<sub>7</sub> → 14NH<sub>3</sub> + 15H<sub>2</sub>O + 2N<sub>2</sub> + 6U<sub>3</sub>O<sub>8</sub></center>

__Evaporation Process__

Evaporation is a process in which an element or compound transitions from its liquid state to its gaseous state below the temperature at which it boils. However, the form of drying we will be dealing with in this stage of the process is working with a reactant that has become mostly solid thanks to the precipitation process. Thus, evaporation will mainly target the water vapor that remains attatched to the ammonium diuranate slurry.

The unit operation that is going to be used for this process is known as a falling film evaporator tank. This type of evaporator is a specific type of vertically oriented shell and tube heat exchanger that is used to separate two or more substances with different boiling point temperatures. It acts as most evaporator columns do in the fact that it heavily relies on a desired product stream that is heavy enough to sink to the bottom while the inpurities rise to the top.

However, in falling film evaporators the process liquid is pumped into the top of the column, and is distributed evenly across all of the heating tubes in the heat exchanger. It is very important that the liquid be well distributed so that it evenly flows down the inside walls of each tube. This film of liquid that is descending through the tubes is known as a “falling film” and is where this particular heat exchanger derives its name.

The process can be summerized as filling the evaporator column to its peak volume, adding heat to evaporate it, and then discharging it to the next stage of the process. The overall process of the falling film evaporator is shown below:
![Falling Film Evaporator](falling-film.jpg)
<p style="text-align:center;"><b>Figure 4: Falling Film Evaporator</b></p>


Each evaporator column will be run at a temperature of 350°C. There while be two product streams exiting this column, one being the vapor extracted from the slurry. This is known as the off-gas produced by this evaporation, and it will be sent to be further processed in a seperate Vapor/Liquid seperation column. The second stream is the product containing the ammonium diuranate that now has a lot less vapor content in it. This product will now be fed to the calcination stage of the drying process to be further processed.


__Calcination Process__

The second stage of the drying process is known as calcination. In terms of the overall process it will follow a similar basis to that of evaporation, but at a higher temperature. However, there are some distinguishable differences in calcination since the feed is now mostly a pure solid. Calcination involves the heating of solids to a high temperature for the purpose of removing volatile substances, oxidizing a portion of mass, or rendering them friable. Therefore, this stage in the drying process is sometimes considered a process of absolute purification.

This stage is not similar to the "evaporation" stage because it is a whole different piece of equipment at a much larger temperature. 

This production happens in large molar quantities, and assuming a 100% conversion rate every 100 kgs of ammonium diuranate will result in 90 kgs of triuranium octoxide. High temperature calcining is practiced producing a greater than 94% U<sub>3</sub>O<sub>8</sub> product having high density [3].

Calcination eliminates water completely, and drives off the ammonia, but care must be taken not to sinter the product.

Sintering can significantly reduce nitric acid solubility, which is critical for some refining processes. Several types of drying equipment have been used in the calcining process, typically arranged in the series of dryers that succeed the main calciner tank. These include single or multiple hearth dryers, drum dryers, belt dryers, screw dryers and radiant heat dryers. There also becomes the problem of how to dispose of the ammonia used as a "sweep-gas" to drive the heat transfer in the calciner. As seen in figure 4, the solution to the produced ammonia is to add a form of fluid bed dryer to the calciner tank to extract the ammonia vapor.

__Sizing and Cost__

The size of each of the main columns used in this process will prove to be vital to the overall drying module. For evaporation, it was determined that there would be 2 columns used in this process. Each of these columns would be sized at 200m<sup>3</sup>. For calcination, it was determined that 2 MHF units would be utilized. Each of these calciner unites would have an overall volume of 350m<sup>3</sup>.

The overall cost of this operation will be strictly based on the main unit operations involved in the process. Thus, it will mainly pertain to the evapotator and calciner columns. For the evaporator tank, it will utilize a stainless steel material to withstand the high temperatures the system is going to be exposed to. Using (x), it is assumed that the cost of each of the 200m<sup>3</sup> falling film evaporator columns using this material will be 28,700 dollars. For the calciner, it will also utilize stainless steel, but will also include an interior refactory brick layer since it will have a low thermal conductivity during the heat reaction. This in turn will not only lead to protection of the stainless steel outer layer, but will also yield greater energy efficiency for the overall heating process. Using (x), each of the MHF calciner units, with a capacity of 350m<sup>3</sup>, will cost 75,370 dollars. 

In total, the overall drying process will cost approximately 208,140 dollars to construct and design.

__Modeling the Module__

To model this process, the process needs to remain seperated by the two main stages of drying. Thus, two seperate subsections to this module will be created for both evaporation and calcination.

Starting with evaporation, this basis of the model needed some input parameters to work with. The feed mass flowrate was determined to be 5800 kg/min, along with a  set overall volume of 400m<sup>3</sup> (remember there are two tanks). The third and final parameter that will be monitored is the product mass density, which is being feed in at 

a paramter had to be selected to base the model after that would be able to monitor the amount of flow entering and exiting the tank. This was chosen to be the liquid volume of the evaporator tank, but it came with some conflicts. Being able to differentiate whether the tank was filling, evaporating, or discharging into the next stage was difficult to determine using only this parameter. 
